## Importing Libraries

In [0]:
import warnings
warnings.filterwarnings("ignore")
import os
from google.colab import drive
import tensorflow as tf
import pandas as pd
import numpy as np
import zipfile
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import Ridge,BayesianRidge,Lasso 
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score,precision_score,recall_score,mean_squared_error
from string import ascii_letters
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
!pip3 install rouge==0.3.1 
from rouge import Rouge

### Mounting Drive

In [4]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

In [6]:
!ls 'gdrive/My Drive/ASAG Work/All Data'

 bm25data.tsv		       Mohler_till_POSNew.csv
 bm25.tsv		       moh_pairs.txt
 Choice-0.csv		       numberbatch-en-17.02.txt
 Choice-1.csv		       Paper2.csv
 Choice-2.csv		       Paper3.csv
 Choice-3.csv		       Paper5.csv
 Choice-4.csv		       SemEval
 Choice-5.csv		       SemEval_Cleaned.csv
 Choice-6.csv		      'SemEval Full Data.csv'
 elmo_semeval_embeddings.csv   SemEval_Question_Infersent_Embeddings.csv
'Feature Selection.csv'        SemEval_Ref_Ans_Infersent_Embeddings.csv
'Feature Selection.gsheet'     SemEval_Rouge.csv
 glove.6B.100d.txt	       SemEval_Stu_Ans_Infersent_Embeddings.csv
 glove.6B.100d.txt.word2vec    SemEval_till_POS.csv
 mini.h5		       SemEval_till_POSNew.csv
 Mohler1_ans.pickle	       SemEval_with_Jaccard.csv
 Mohler1_ques.pickle	       ShortAnswerGrading_v1.0
 MohlerFullData.csv	       ShortAnswerGrading_v2.0
 Mohler_till_POS.csv


## Loading Data

In [0]:
data = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_till_POSNew.csv",sep=',')
j = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_with_Jaccard.csv",sep=',')
r = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_Rouge.csv",sep=',')
t = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/bm25.tsv",sep='\t')
data['bm25'] = t['0']
data['Jaccard'] = j['Jaccard']
data = pd.concat([data,r],axis = 1)

In [0]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'qid', 'ref_id', 'question', 'ref_answer',
       'stu_answer', 'train/test', 'accuracy', 'q_word_count', 'q_char_count',
       'q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
       's_word_count', 's_char_count', 's_avg_word', 'Row_no', 'q_nouns',
       'q_adjectives', 'q_verbs', 'q_nouns_vs_length',
       'q_adjectives_vs_length', 'q_verbs_vs_length', 'q_nouns_vs_words',
       'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
       'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
       'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
       'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
       's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
       's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
       'precision', 'recall', 'F1_score', 'how_flag', 'what_flag', 'why_flag',
       'who_flag', 'which_flag', 'when_flag', 'where_flag', 'wh

In [0]:
all_features = ['q_word_count', 'q_char_count',
       'q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
       's_word_count', 's_char_count', 's_avg_word', 'q_nouns',
       'q_adjectives', 'q_verbs', 'q_nouns_vs_length',
       'q_adjectives_vs_length', 'q_verbs_vs_length', 'q_nouns_vs_words',
       'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
       'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
       'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
       'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
       's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
       's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
       'precision', 'recall', 'F1_score', 'how_flag', 'what_flag', 'why_flag',
       'who_flag', 'which_flag', 'when_flag', 'where_flag', 'whom_flag',
       'bm25', 'Jaccard', 'r1_f', 'r1_p', 'r1_r', 'r2_f', 'r2_p', 'r2_r',
       'rlcs_f', 'rlcs_p', 'rlcs_r', 's_verbs_vs_r_verbs',
       's_nouns_vs_r_nouns', 's_adjectives_vs_r_adjectives',
       's_word_count_vs_r_word_count', 's_nouns_vs_words_vs_r_nouns_vs_words',
       's_verbs_vs_words_vs_r_verbs_vs_words',
       's_adjectives_vs_words_vs_r_adjectives_vs_words', 'rs_word_diff',
       'rs_noun_vs_words_diff', 'rs_verb_vs_words_diff',
       'rs_adjectives_vs_words_diff']

best = ['q_word_count', 'q_char_count',
       'q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
       's_word_count', 's_char_count', 's_avg_word', 'q_nouns',
       'q_adjectives', 'q_verbs', 'q_nouns_vs_length',
       'q_adjectives_vs_length', 'q_verbs_vs_length', 'q_nouns_vs_words',
       'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
       'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
       'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
       'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
       's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
       's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
       'precision', 'recall', 'F1_score','bm25', 'Jaccard', 'r1_f', 'r1_p', 'r1_r', 'r2_f', 'r2_p', 'r2_r',
       'rlcs_f', 'rlcs_p', 'rlcs_r']


label = 'accuracy'

In [0]:
def get_data(data,features,label,n_classes,est = 500):
    
    # weighted f1 for all 3 followed by macro f1
    sota2=pd.DataFrame([('sota',0.791,0.7478,0.7178,0.7858,0.7385,0.7089)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    sota3=pd.DataFrame([('sota',0.7143,0.6362,0.6115,0.6662,0.4912,0.4794)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    sota5=pd.DataFrame([('sota',0.642,0.4881,0.4923,0.4808,0.3763,0.3574)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    
    our2=pd.DataFrame([('our_best',0.798629,0.77561,0.760224,0.798629,0.77561,0.760224)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    our3=pd.DataFrame([('our_best',0.663615,0.556199,0.593633,0.631032,0.507533,0.435268)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    our5=pd.DataFrame([('our_best',0.592112,0.505764,0.441613,0.582376,0.496443,0.364422)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    
    def get_req_data(tr_ts):
        indices = data[(data['train/test'].str.contains(tr_ts)) & (data['train/test'].str.contains(str(n_classes)))].index.tolist()
        d = data.loc[indices,:].reset_index(drop=True)
        f = d.loc[:,features]
        y = d.loc[:,label]
        return f,y
    
    # run if no embeddings loaded
    f_train,y_train = get_req_data('train')
    f_test1,y_test1 = get_req_data('unseen-answers')
    f_test2,y_test2 = get_req_data('unseen-questions')
    f_test3,y_test3 = get_req_data('unseen-domains')
    
    # convert labels to ids
    un_acc = y_train.unique()
    lab_to_int, int_to_lab = {},{}
    for n,u in enumerate(un_acc):
        lab_to_int[u] = n
        int_to_lab[n] = u

    y_train = y_train.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
    y_test1 = y_test1.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
    y_test2 = y_test2.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
    y_test3 = y_test3.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
    
  
    
    
    return f_train, y_train, f_test1, y_test1, f_test2, y_test2, f_test3, y_test3

In [0]:
features = best
n_classes = 2
f_train, y_train, f_test1, y_test1, f_test2, y_test2, f_test3, y_test3 = get_data(data,features,label,n_classes)

In [0]:
  est = 400
  rfc = RandomForestClassifier(n_estimators=est,criterion='entropy')
  etc = ExtraTreesClassifier(n_estimators=est,criterion='entropy')
  gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.01,n_estimators=est)
  gmb = xgb.XGBClassifier(n_estimators = est, )
  rfc.fit(f_train,y_train)
  etc.fit(f_train,y_train)
  gbc.fit(f_train,y_train)
  gmb.fit(f_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=400,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
def predict(model, f_test, y_test, n_classes):
         

        y_pred = model.predict(f_test)
        if n_classes == 2:
            f1 = f1_score(y_test,y_pred,labels=list(int_to_lab.keys()),average='binary')
            pr = precision_score(y_test,y_pred,labels=list(int_to_lab.keys()),average='binary')
            re = recall_score(y_test,y_pred,labels=list(int_to_lab.keys()),average='binary')
        else:
            f1 = f1_score(y_test,y_pred,labels=list(int_to_lab.keys()),average=method)
            pr = precision_score(y_test,y_pred,labels=list(int_to_lab.keys()),average=method)
            re = recall_score(y_test,y_pred,labels=list(int_to_lab.keys()),average=method)
        print(pr,re,f1)

In [0]:
predict(etc,f_test1,y_test1,n_classes)

NameError: ignored

## Extra Features

### Extra POS

In [0]:
def fab(row, types):
    if(row['r_'+types]!=0):
        return row['s_'+types]/row['r_'+ types]
    else:
        return 0

In [0]:
data['s_verbs_vs_r_verbs'] = data.apply(lambda x : fab(x,"verbs"), axis = 1)
data['s_nouns_vs_r_nouns'] = data.apply(lambda x : fab(x,"nouns"), axis = 1)
data['s_adjectives_vs_r_adjectives'] = data.apply(lambda x : fab(x,"adjectives"), axis = 1)
data['s_word_count_vs_r_word_count'] = data['s_word_count']/data['r_word_count']
data['s_nouns_vs_words_vs_r_nouns_vs_words'] = data.apply(lambda x : fab(x, "nouns_vs_words"), axis = 1)
data['s_verbs_vs_words_vs_r_verbs_vs_words'] = data.apply(lambda x : fab(x, "verbs_vs_words"), axis = 1)
data['s_adjectives_vs_words_vs_r_adjectives_vs_words'] = data.apply(lambda x : fab(x, "adjectives_vs_words"), axis = 1)

In [0]:
data['rs_word_diff'] = data['r_word_count'] - data['s_word_count']
data['rs_noun_vs_words_diff'] = data['r_nouns_vs_words'] - data['s_nouns_vs_words']
data['rs_verb_vs_words_diff'] = data['r_verbs_vs_words'] - data['s_verbs_vs_words']
data['rs_adjectives_vs_words_diff'] = data['r_adjectives_vs_words'] - data['s_adjectives_vs_words']

### Rogue

In [7]:
data = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/Mohler_till_POSNew.csv",sep=',')
data.head()

,Unnamed: 0,Question,Ref_Answer,Answer,Score,Q_word_count,Q_char_count,Q_avg_word,R_word_count,R_char_count,...,recall,F1_score,how_flag,what_flag,why_flag,who_flag,which_flag,when_flag,where_flag,whom_flag
0,0,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,high risk problems be address in the prototype...,3.5,12,61,4.454545,12,72,...,0.333333,0.037037,False,True,False,False,False,False,False,False
1,1,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,to simulate portion of the desire final produc...,5.0,12,61,4.454545,12,72,...,0.750000,0.156977,False,True,False,False,False,False,False,False
2,2,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,a prototype program simulate the behaviors of ...,4.0,12,61,4.454545,12,72,...,0.916667,0.530702,False,True,False,False,False,False,False,False
3,3,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,define in the specification phase a prototype ...,5.0,12,61,4.454545,12,72,...,0.833333,0.193798,False,True,False,False,False,False,False,False
4,4,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,it be use to let the users have a first idea o...,3.0,12,61,4.454545,12,72,...,0.500000,0.075000,False,True,False,False,False,False,False,False


In [10]:
def get_Rogue(df,columns):
  rouge = Rouge()
  l1=[]
  l2=[]
  for i in df.index:
    l1.append(df[columns[0]][i])
    l2.append(df[columns[1]][i])
    if(i%10000 == 0):
      print(i)
  return rouge.get_scores(l1,l2)    

#sim_columns = ['ref_answer','stu_answer']
sim_columns = ['Ref_Answer','Answer']
scores = get_Rogue(data,sim_columns)

0


In [12]:
r1 = pd.DataFrame(scores)['rouge-1'].apply(pd.Series)
r2 = pd.DataFrame(scores)['rouge-2'].apply(pd.Series)
r3 = pd.DataFrame(scores)['rouge-l'].apply(pd.Series)
r = pd.concat([r1,r2,r3] , axis = 1, )
r.columns = ['r1_f','r1_p','r1_r','r2_f','r2_p','r2_r','rlcs_f','rlcs_p','rlcs_r']
r.to_csv("gdrive/My Drive/ASAG Work/All Data/Mohler_Rouge.csv",index = False)
r.head()

,r1_f,r1_p,r1_r,r2_f,r2_p,r2_r,rlcs_f,rlcs_p,rlcs_r
0,0.187500,0.333333,0.130435,0.000000,0.0,0.000000,0.141904,0.333333,0.130435
1,0.341463,0.777778,0.218750,0.160000,0.4,0.100000,0.230916,0.777778,0.218750
2,0.640000,0.888889,0.500000,0.518519,0.7,0.411765,0.488912,0.777778,0.437500
3,0.368421,0.777778,0.241379,0.244898,0.6,0.153846,0.220240,0.666667,0.206897
4,0.210526,0.444444,0.137931,0.043478,0.1,0.027778,0.110120,0.333333,0.103448


In [0]:
def get_Jaccard(df,columns):
  df['Jaccard'] = df['Q_avg_word']
  for i in df.index:
    a = set(df[columns[0]][i].split()) 
    b = set(df[columns[1]][i].split())
    c = a.intersection(b)
    df['Jaccard'][i] = float(len(c)) / (len(a) + len(b) - len(c))
  return df  

In [16]:
#sim_columns = ['ref_answer','stu_answer']
temp = get_Jaccard(data,sim_columns)
temp.head()

,Unnamed: 0,Question,Ref_Answer,Answer,Score,Q_word_count,Q_char_count,Q_avg_word,R_word_count,R_char_count,...,F1_score,how_flag,what_flag,why_flag,who_flag,which_flag,when_flag,where_flag,whom_flag,Jaccard
0,0,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,high risk problems be address in the prototype...,3.5,12,61,4.454545,12,72,...,0.037037,False,True,False,False,False,False,False,False,0.133333
1,1,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,to simulate portion of the desire final produc...,5.0,12,61,4.454545,12,72,...,0.156977,False,True,False,False,False,False,False,False,0.228571
2,2,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,a prototype program simulate the behaviors of ...,4.0,12,61,4.454545,12,72,...,0.530702,False,True,False,False,False,False,False,False,0.500000
3,3,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,define in the specification phase a prototype ...,5.0,12,61,4.454545,12,72,...,0.193798,False,True,False,False,False,False,False,False,0.250000
4,4,what be the role of a prototype program in pro...,to simulate the behaviour of portion of the de...,it be use to let the users have a first idea o...,3.0,12,61,4.454545,12,72,...,0.075000,False,True,False,False,False,False,False,False,0.142857


In [0]:
temp.to_csv("gdrive/My Drive/ASAG Work/All Data/Mohler_with_Jaccard.csv",index = False)

## Features Definiton

In [0]:
data.columns

In [0]:
q_b_f = ['q_word_count','q_char_count','q_avg_word']
a_b_f = ['r_word_count','r_char_count','r_avg_word','s_word_count','s_char_count','s_avg_word']
q_pos = ['q_nouns','q_adjectives','q_verbs','q_nouns_vs_length','q_adjectives_vs_length','q_verbs_vs_length','q_nouns_vs_words','q_adjectives_vs_words','q_verbs_vs_words']
a_pos = ['r_nouns','r_adjectives','r_verbs','r_nouns_vs_length','r_adjectives_vs_length','r_verbs_vs_length','r_nouns_vs_words','r_adjectives_vs_words','r_verbs_vs_words',
         's_nouns','s_adjectives','s_verbs','s_nouns_vs_length','s_adjectives_vs_length','s_verbs_vs_length','s_nouns_vs_words','s_adjectives_vs_words','s_verbs_vs_words']
similarity = ['Jaccard','bm25','r1_f','r1_p','r1_r','r2_f','r2_p','r2_r','rlcs_f','rlcs_p','rlcs_r']
new_pos = ['s_verbs_vs_r_verbs','s_nouns_vs_r_nouns','s_adjectives_vs_r_adjectives','s_word_count_vs_r_word_count',
           's_nouns_vs_words_vs_r_nouns_vs_words','s_verbs_vs_words_vs_r_verbs_vs_words','s_adjectives_vs_words_vs_r_adjectives_vs_words',
           'rd_word_diff','rs_noun_vs_words_diff','rs_verb_vs_words_diff','rs_adjectives_vs_words_diff']


# Mohler

In [0]:
data = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/Mohler_till_POSNew.csv",sep=',')

In [0]:
features = ['Q_word_count', 'Q_char_count', 'Q_avg_word', 'R_word_count',
       'R_char_count', 'R_avg_word', 'A_word_count', 'A_char_count',
       'A_avg_word', 'Q_nouns', 'Q_adjectives', 'Q_verbs',
       'Q_nouns_vs_length', 'Q_adjectives_vs_length', 'Q_verbs_vs_length',
       'Q_nouns_vs_words', 'Q_adjectives_vs_words', 'Q_verbs_vs_words',
       'R_nouns', 'R_adjectives', 'R_verbs', 'R_nouns_vs_length',
       'R_adjectives_vs_length', 'R_verbs_vs_length', 'R_nouns_vs_words',
       'R_adjectives_vs_words', 'R_verbs_vs_words', 'A_nouns', 'A_adjectives',
       'A_verbs', 'A_nouns_vs_length', 'A_adjectives_vs_length',
       'A_verbs_vs_length', 'A_nouns_vs_words', 'A_adjectives_vs_words',
       'A_verbs_vs_words', 'precision', 'recall', 'F1_score', 'how_flag',
       'what_flag', 'why_flag', 'who_flag', 'which_flag', 'when_flag',
       'where_flag', 'whom_flag']

label = 'Score'

In [0]:
split = int(0.7*data.shape[0])
f_train,f_test = data.loc[:split,features], data.loc[split:,features]
y_train,y_test = data.loc[:split,label], data.loc[split:,label]

In [0]:
breg = BayesianRidge(normalize=True)
rfr = RandomForestRegressor(n_estimators=500,criterion='mse')
etr = ExtraTreesRegressor(n_estimators=500,criterion='mse')
gbr = GradientBoostingRegressor(n_estimators=500,loss='huber',learning_rate=0.1)

In [0]:
breg.fit(f_train,y_train)
rfr.fit(f_train,y_train)
etr.fit(f_train,y_train)
gbr.fit(f_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='huber', max_depth=3,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [0]:
y_pred = breg.predict(f_test)
print("RMSE : %.3f \t Corr : %.3f"%(np.sqrt(mean_squared_error(y_pred,y_test)),np.corrcoef(y_test,y_pred)[0,1]))
y_pred = rfr.predict(f_test)
print("RMSE : %.3f \t Corr : %.3f"%(np.sqrt(mean_squared_error(y_pred,y_test)),np.corrcoef(y_test,y_pred)[0,1]))
y_pred = etr.predict(f_test)
print("RMSE : %.3f \t Corr : %.3f"%(np.sqrt(mean_squared_error(y_pred,y_test)),np.corrcoef(y_test,y_pred)[0,1]))
y_pred = gbr.predict(f_test)
print("RMSE : %.3f \t Corr : %.3f"%(np.sqrt(mean_squared_error(y_pred,y_test)),np.corrcoef(y_test,y_pred)[0,1]))

RMSE : 0.982 	 Corr : 0.545
RMSE : 0.715 	 Corr : 0.802
RMSE : 0.644 	 Corr : 0.842
RMSE : 0.780 	 Corr : 0.748


# Tree Models

## Last Result Experimentation

### Function

In [0]:
def get_required_data_and_results(data,features,label,n_classes,est = 500):
    
    # weighted f1 for all 3 followed by macro f1
    sota2=pd.DataFrame([('sota',0.791,0.7478,0.7178,0.7858,0.7385,0.7089)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    sota3=pd.DataFrame([('sota',0.7143,0.6362,0.6115,0.6662,0.4912,0.4794)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    sota5=pd.DataFrame([('sota',0.642,0.4881,0.4923,0.4808,0.3763,0.3574)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    
    our2=pd.DataFrame([('our_best',0.798629,0.77561,0.760224,0.798629,0.77561,0.760224)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    our3=pd.DataFrame([('our_best',0.663615,0.556199,0.593633,0.631032,0.507533,0.435268)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    our5=pd.DataFrame([('our_best',0.592112,0.505764,0.441613,0.582376,0.496443,0.364422)],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    
    def get_req_data(tr_ts):
        indices = data[(data['train/test'].str.contains(tr_ts)) & (data['train/test'].str.contains(str(n_classes)))].index.tolist()
        d = data.loc[indices,:].reset_index(drop=True)
        f = d.loc[:,features]
        y = d.loc[:,label]
        return f,y
    
    # run if no embeddings loaded
    f_train,y_train = get_req_data('train')
    f_test1,y_test1 = get_req_data('unseen-answers')
    f_test2,y_test2 = get_req_data('unseen-questions')
    f_test3,y_test3 = get_req_data('unseen-domains')
    
    # convert labels to ids
    un_acc = y_train.unique()
    lab_to_int, int_to_lab = {},{}
    for n,u in enumerate(un_acc):
        lab_to_int[u] = n
        int_to_lab[n] = u

    y_train = y_train.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
    y_test1 = y_test1.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
    y_test2 = y_test2.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
    y_test3 = y_test3.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
    
    def run_tree(model,method):
        results = []

        y_pred = model.predict(f_test1)
        if n_classes == 2:
            f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
            pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
            re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        else:
            f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
            pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
            re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        results.extend([pr,re,f1])

        y_pred = model.predict(f_test2)
        if n_classes == 2:
            f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
            pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
            re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        else:
            f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
            pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
            re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        results.extend([pr,re,f1])

        y_pred = model.predict(f_test3)
        if n_classes == 2:
            f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
            pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
            re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        else:
            f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
            pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
            re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        results.extend([pr,re,f1])

        return [results[2],results[5],results[8]]
    
    def res_diff(res,grid):
        if n_classes == 2:
            grid = grid.append(sota2,ignore_index=True)
            grid = grid.append(our2,ignore_index=True)
            grid = grid.append(pd.DataFrame(res,columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1']),ignore_index=True)
            return grid
        elif n_classes == 3:
            grid = grid.append(sota3,ignore_index=True)
            grid = grid.append(our3,ignore_index=True)
            grid = grid.append(pd.DataFrame(res,columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1']),ignore_index=True)
            return grid
        elif n_classes == 5:
            grid = grid.append(sota5,ignore_index=True)
            grid = grid.append(our5,ignore_index=True)
            grid = grid.append(pd.DataFrame(res,columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1']),ignore_index=True)
            return grid

    rfc = RandomForestClassifier(n_estimators=est,criterion='entropy')
    etc = ExtraTreesClassifier(n_estimators=est,criterion='entropy')
    gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.01,n_estimators=est)
    gmb = xgb.XGBClassifier(n_estimators = est, )
    rfc.fit(f_train,y_train)
    etc.fit(f_train,y_train)
    gbc.fit(f_train,y_train)
    gmb.fit(f_train,y_train)
    
    over_all = []
    grid = pd.DataFrame([],columns=['Model','UA_wf1','UQ_wf1','UD_wf1','UA_mf1','UQ_mf1','UD_mf1'])
    res = run_tree(rfc,'weighted')
    res1 = run_tree(rfc,'macro')
    over_all.append(tuple(['rfc']+res+res1))
    res = run_tree(etc,'weighted')
    res1 = run_tree(etc,'macro')
    over_all.append(tuple(['etc']+res+res1))
    res = run_tree(gbc,'weighted')
    res1 = run_tree(gbc,'macro')
    over_all.append(tuple(['gbc']+res+res1))
    res = run_tree(gmb,'weighted')
    res1 = run_tree(gmb,'macro')
    over_all.append(tuple(['gmb']+res+res1))
    grid = res_diff(over_all,grid)
    
    return [grid,etc]

In [0]:
#our best
features_b = ['q_word_count', 'q_avg_word','q_nouns_vs_words','q_adjectives_vs_words', 'q_verbs_vs_words',
            'r_word_count', 'r_char_count', 'r_avg_word','s_word_count', 's_char_count', 
            's_avg_word', 'r_nouns', 'r_adjectives','r_verbs', 'r_nouns_vs_words', 
            'r_adjectives_vs_words','r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
            's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words','precision', 'recall',
            'F1_score', 'how_flag', 'what_flag', 'why_flag','who_flag', 'which_flag', 'when_flag',
            'where_flag', 'whom_flag','rs_word_diff', 'rs_noun_vs_words_diff', 'rs_verb_vs_words_diff',
            'rs_adjectives_vs_words_diff']


# all features so far
features1 = ['q_word_count','q_char_count','q_avg_word','q_nouns','q_adjectives','q_verbs','q_nouns_vs_length','q_adjectives_vs_length','q_verbs_vs_length','q_nouns_vs_words','q_adjectives_vs_words','q_verbs_vs_words',
            'r_word_count','r_char_count','r_avg_word','r_nouns','r_adjectives','r_verbs','r_nouns_vs_length','r_adjectives_vs_length','r_verbs_vs_length','r_nouns_vs_words','r_adjectives_vs_words','r_verbs_vs_words',
            's_word_count','s_char_count','s_avg_word','s_nouns','s_adjectives','s_verbs','s_nouns_vs_length','s_adjectives_vs_length','s_verbs_vs_length','s_nouns_vs_words','s_adjectives_vs_words','s_verbs_vs_words',
            'precision', 'recall', 'F1_score','bm25',
            'how_flag','what_flag','why_flag','who_flag','which_flag','when_flag','where_flag','whom_flag',
            's_verbs_vs_r_verbs','s_nouns_vs_r_nouns','s_adjectives_vs_r_adjectives','s_word_count_vs_r_word_count','s_nouns_vs_words_vs_r_nouns_vs_words','s_verbs_vs_words_vs_r_verbs_vs_words','s_adjectives_vs_words_vs_r_adjectives_vs_words','rs_word_diff','rs_noun_vs_words_diff','rs_verb_vs_words_diff','rs_adjectives_vs_words_diff']

# no basic features
features2 = ['q_nouns','q_adjectives','q_verbs','q_nouns_vs_length','q_adjectives_vs_length','q_verbs_vs_length','q_nouns_vs_words','q_adjectives_vs_words','q_verbs_vs_words',
             'r_nouns','r_adjectives','r_verbs','r_nouns_vs_length','r_adjectives_vs_length','r_verbs_vs_length','r_nouns_vs_words','r_adjectives_vs_words','r_verbs_vs_words',
             's_nouns','s_adjectives','s_verbs','s_nouns_vs_length','s_adjectives_vs_length','s_verbs_vs_length','s_nouns_vs_words','s_adjectives_vs_words','s_verbs_vs_words',
             'precision', 'recall', 'F1_score',
             'how_flag','what_flag','why_flag','who_flag','which_flag','when_flag','where_flag','whom_flag',
             's_verbs_vs_r_verbs','s_nouns_vs_r_nouns','s_adjectives_vs_r_adjectives','s_word_count_vs_r_word_count','s_nouns_vs_words_vs_r_nouns_vs_words','s_verbs_vs_words_vs_r_verbs_vs_words','s_adjectives_vs_words_vs_r_adjectives_vs_words','rs_word_diff','rs_noun_vs_words_diff','rs_verb_vs_words_diff','rs_adjectives_vs_words_diff']

# no question based features
features3 = ['r_word_count','r_char_count','r_avg_word','r_nouns','r_adjectives','r_verbs','r_nouns_vs_length','r_adjectives_vs_length','r_verbs_vs_length','r_nouns_vs_words','r_adjectives_vs_words','r_verbs_vs_words',
             's_word_count','s_char_count','s_avg_word','s_nouns','s_adjectives','s_verbs','s_nouns_vs_length','s_adjectives_vs_length','s_verbs_vs_length','s_nouns_vs_words','s_adjectives_vs_words','s_verbs_vs_words',
             'precision', 'recall', 'F1_score',
             's_verbs_vs_r_verbs','s_nouns_vs_r_nouns','s_adjectives_vs_r_adjectives','s_word_count_vs_r_word_count','s_nouns_vs_words_vs_r_nouns_vs_words','s_verbs_vs_words_vs_r_verbs_vs_words','s_adjectives_vs_words_vs_r_adjectives_vs_words','rs_word_diff','rs_noun_vs_words_diff','rs_verb_vs_words_diff','rs_adjectives_vs_words_diff']

# no advanced POS based features
features4 = ['q_word_count','q_char_count','q_avg_word','q_nouns','q_adjectives','q_verbs',
             'r_word_count','r_char_count','r_avg_word','r_nouns','r_adjectives','r_verbs',
             's_word_count','s_char_count','s_avg_word','s_nouns','s_adjectives','s_verbs',
             'precision', 'recall', 'F1_score',
             'how_flag','what_flag','why_flag','who_flag','which_flag','when_flag','where_flag','whom_flag']

# no IBM word similarity features
features5 = ['q_word_count','q_char_count','q_avg_word','q_nouns','q_adjectives','q_verbs','q_nouns_vs_length','q_adjectives_vs_length','q_verbs_vs_length','q_nouns_vs_words','q_adjectives_vs_words','q_verbs_vs_words',
            'r_word_count','r_char_count','r_avg_word','r_nouns','r_adjectives','r_verbs','r_nouns_vs_length','r_adjectives_vs_length','r_verbs_vs_length','r_nouns_vs_words','r_adjectives_vs_words','r_verbs_vs_words',
            's_word_count','s_char_count','s_avg_word','s_nouns','s_adjectives','s_verbs','s_nouns_vs_length','s_adjectives_vs_length','s_verbs_vs_length','s_nouns_vs_words','s_adjectives_vs_words','s_verbs_vs_words',
            'how_flag','what_flag','why_flag','who_flag','which_flag','when_flag','where_flag','whom_flag',
            's_verbs_vs_r_verbs','s_nouns_vs_r_nouns','s_adjectives_vs_r_adjectives','s_word_count_vs_r_word_count','s_nouns_vs_words_vs_r_nouns_vs_words','s_verbs_vs_words_vs_r_verbs_vs_words','s_adjectives_vs_words_vs_r_adjectives_vs_words','rs_word_diff','rs_noun_vs_words_diff','rs_verb_vs_words_diff','rs_adjectives_vs_words_diff']

label = 'accuracy'

###  Results

In [0]:
all_features = ['q_word_count', 'q_char_count',
       'q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
       's_word_count', 's_char_count', 's_avg_word', 'q_nouns',
       'q_adjectives', 'q_verbs', 'q_nouns_vs_length',
       'q_adjectives_vs_length', 'q_verbs_vs_length', 'q_nouns_vs_words',
       'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
       'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
       'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
       'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
       's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
       's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
       'precision', 'recall', 'F1_score', 'how_flag', 'what_flag', 'why_flag',
       'who_flag', 'which_flag', 'when_flag', 'where_flag', 'whom_flag',
       'bm25', 'Jaccard', 'r1_f', 'r1_p', 'r1_r', 'r2_f', 'r2_p', 'r2_r',
       'rlcs_f', 'rlcs_p', 'rlcs_r', 's_verbs_vs_r_verbs',
       's_nouns_vs_r_nouns', 's_adjectives_vs_r_adjectives',
       's_word_count_vs_r_word_count', 's_nouns_vs_words_vs_r_nouns_vs_words',
       's_verbs_vs_words_vs_r_verbs_vs_words',
       's_adjectives_vs_words_vs_r_adjectives_vs_words', 'rs_word_diff',
       'rs_noun_vs_words_diff', 'rs_verb_vs_words_diff',
       'rs_adjectives_vs_words_diff']

best = ['q_word_count', 'q_char_count',
       'q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
       's_word_count', 's_char_count', 's_avg_word', 'q_nouns',
       'q_adjectives', 'q_verbs', 'q_nouns_vs_length',
       'q_adjectives_vs_length', 'q_verbs_vs_length', 'q_nouns_vs_words',
       'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
       'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
       'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
       'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
       's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
       's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
       'precision', 'recall', 'F1_score','bm25', 'Jaccard', 'r1_f', 'r1_p', 'r1_r', 'r2_f', 'r2_p', 'r2_r',
       'rlcs_f', 'rlcs_p', 'rlcs_r']

# best = ['q_word_count', 'q_char_count',
#        'q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
#        's_word_count', 's_char_count', 's_avg_word', 'q_nouns',
#        'q_adjectives', 'q_verbs', 'q_nouns_vs_length',
#        'q_adjectives_vs_length', 'q_verbs_vs_length', 'q_nouns_vs_words',
#        'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
#        'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
#        'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
#        'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
#        's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
#        's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
#        'precision', 'recall', 'F1_score', 'how_flag', 'what_flag', 'why_flag',
#        'who_flag', 'which_flag', 'when_flag', 'where_flag', 'whom_flag']
label = 'accuracy'

In [0]:
q_bf = ['q_word_count','q_char_count','q_avg_word']
a_bf = ['r_word_count','r_char_count','r_avg_word','s_word_count','s_char_count','s_avg_word']
q_pos = ['q_nouns','q_adjectives','q_verbs','q_nouns_vs_length','q_adjectives_vs_length','q_verbs_vs_length','q_nouns_vs_words','q_adjectives_vs_words','q_verbs_vs_words']
a_pos = ['r_nouns','r_adjectives','r_verbs','r_nouns_vs_length','r_adjectives_vs_length','r_verbs_vs_length','r_nouns_vs_words','r_adjectives_vs_words','r_verbs_vs_words',
         's_nouns','s_adjectives','s_verbs','s_nouns_vs_length','s_adjectives_vs_length','s_verbs_vs_length','s_nouns_vs_words','s_adjectives_vs_words','s_verbs_vs_words']
similarity = ['Jaccard','bm25','r1_f','r1_p','r1_r','r2_f','r2_p','r2_r','rlcs_f','rlcs_p','rlcs_r']
paper = ['precision','recall','F1_score']
new_pos = ['s_verbs_vs_r_verbs','s_nouns_vs_r_nouns','s_adjectives_vs_r_adjectives','s_word_count_vs_r_word_count',
           's_nouns_vs_words_vs_r_nouns_vs_words','s_verbs_vs_words_vs_r_verbs_vs_words','s_adjectives_vs_words_vs_r_adjectives_vs_words',
           'rd_word_diff','rs_noun_vs_words_diff','rs_verb_vs_words_diff','rs_adjectives_vs_words_diff']
flags = ['how_flag','what_flag','why_flag','who_flag','which_flag','when_flag','where_flag','whom_flag']


In [0]:
choices = [ a_bf+a_pos+similarity+paper,
            a_bf+a_pos+similarity+paper+flags,
            a_bf+a_pos+similarity+paper+q_pos,
            a_bf+a_pos+similarity+paper+q_pos+flags,
            similarity+paper+a_pos+q_pos,
            all_features,
            similarity+paper]

In [0]:
for choice in range(len(choices)):
  features = choices[choice]
  d = pd.concat( [get_required_data_and_results(data,features,label,2) , get_required_data_and_results(data,features,label,3), get_required_data_and_results(data,features,label,5)] , axis = 1) 
  print(d)
  d.to_csv("gdrive/My Drive/ASAG Work/All Data/Choice-"+(str)(choice)+".csv",index = False)

      Model    UA_wf1    UQ_wf1    UD_wf1    UA_mf1    UQ_mf1    UD_mf1  \
0      sota  0.791000  0.747800  0.717800  0.785800  0.738500  0.708900   
1  our_best  0.798629  0.775610  0.760224  0.798629  0.775610  0.760224   
2       rfc  0.821429  0.787202  0.757576  0.821429  0.787202  0.757576   
3       etc  0.810580  0.782565  0.757902  0.810580  0.782565  0.757902   
4       gbc  0.788686  0.788100  0.760244  0.788686  0.788100  0.760244   
5       gmb  0.805743  0.769719  0.742336  0.805743  0.769719  0.742336   

      Model    UA_wf1    UQ_wf1    ...       UA_mf1    UQ_mf1    UD_mf1  \
0      sota  0.714300  0.636200    ...     0.666200  0.491200  0.479400   
1  our_best  0.663615  0.556199    ...     0.631032  0.507533  0.435268   
2       rfc  0.662968  0.549922    ...     0.619328  0.499230  0.436187   
3       etc  0.652371  0.540785    ...     0.612250  0.483437  0.440044   
4       gbc  0.593248  0.556969    ...     0.529613  0.491340  0.470896   
5       gmb  0.650509  0

In [0]:
# pd.concat( [get_required_data_and_results(data,features,label,2) , get_required_data_and_results(data,features,label,3), get_required_data_and_results(data,features,label,5)] , axis = 1)

In [0]:
features = best
n_classes = 2
etc = get_required_data_and_results(data,features,label,n_classes)[1]

In [0]:
data.head()

,Unnamed: 0,Unnamed: 0.1,qid,ref_id,question,ref_answer,stu_answer,train/test,accuracy,q_word_count,...,Jaccard,r1_f,r1_p,r1_r,r2_f,r2_p,r2_r,rlcs_f,rlcs_p,rlcs_r
0,0,0,BULB_C_VOLTAGE_EXPLAIN_WHY1,answer204,explain why you get a voltage read of # # for ...,terminal # and the positive terminal be separa...,positive battery terminal be separate by a gap...,training_2way,correct,16,...,0.583333,0.736842,0.777778,0.700000,0.400000,0.400,0.400000,0.628109,0.666667,0.600000
1,1,1,BULB_C_VOLTAGE_EXPLAIN_WHY1,answer205,explain why you get a voltage read of # # for ...,terminal # and the positive terminal be not co...,terminal # be not connect to the positive term...,training_2way,correct,16,...,0.777778,0.875000,0.875000,0.875000,0.625000,0.625,0.625000,0.500000,0.500000,0.500000
2,2,2,BULB_C_VOLTAGE_EXPLAIN_WHY1,answer204,explain why you get a voltage read of # # for ...,terminal # and the positive terminal be separa...,because terminal # be connect to the positive ...,training_2way,incorrect,16,...,0.384615,0.555556,0.555556,0.555556,0.210526,0.200,0.222222,0.444444,0.444444,0.444444
3,3,3,BULB_C_VOLTAGE_EXPLAIN_WHY1,answer204,explain why you get a voltage read of # # for ...,terminal # and the positive terminal be separa...,because terminal # be not seperated by any gap,training_2way,incorrect,16,...,0.384615,0.555556,0.555556,0.555556,0.111111,0.100,0.125000,0.555556,0.555556,0.555556
4,4,4,BULB_C_VOLTAGE_EXPLAIN_WHY1,answer204,explain why you get a voltage read of # # for ...,terminal # and the positive terminal be separa...,because terminal one be connect to both the ne...,training_2way,incorrect,16,...,0.312500,0.476190,0.555556,0.416667,0.000000,0.000,0.000000,0.274725,0.333333,0.250000


In [0]:
n_classes = 3
get_required_data_and_results(data,features,label,n_classes)

,Model,UA_wf1,UQ_wf1,UD_wf1,UA_mf1,UQ_mf1,UD_mf1
0,sota,0.714300,0.636200,0.611500,0.666200,0.491200,0.479400
1,our_best,0.663615,0.556199,0.593633,0.631032,0.507533,0.435268
2,rfc,0.595159,0.522876,0.557438,0.544870,0.464106,0.429166
3,etc,0.588459,0.514627,0.534930,0.545124,0.461244,0.414776
4,gbc,0.569793,0.500977,0.588203,0.504296,0.418853,0.434036
5,gmb,0.592947,0.514011,0.578718,0.531174,0.439758,0.436496


In [0]:
n_classes = 5
get_required_data_and_results(data,features,label,n_classes)

,Model,UA_wf1,UQ_wf1,UD_wf1,UA_mf1,UQ_mf1,UD_mf1
0,sota,0.642000,0.488100,0.492300,0.480800,0.376300,0.357400
1,our_best,0.592112,0.505764,0.441613,0.582376,0.496443,0.364422
2,rfc,0.489622,0.420543,0.425266,0.361843,0.293919,0.271195
3,etc,0.480560,0.418362,0.420442,0.356497,0.292173,0.270123
4,gbc,0.475207,0.413004,0.450706,0.342243,0.286765,0.282689
5,gmb,0.484321,0.424817,0.442824,0.355712,0.293023,0.280090


## Older Code

In [0]:
def run_tree(model,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,method,to3=False):
    results = []

    y_pred = model.predict(f_test1)
    if n_classes == 2:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    y_pred = model.predict(f_test2)
    if n_classes == 2:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])

    y_pred = model.predict(f_test3)
    if n_classes == 2:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
#     return (results[2],results[5],results[8])
    return results

## Normal Run

In [0]:
# 2 way results
rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=500)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))

-------Weighted--------------
(0.7959183673469388, 0.7727498756837393, 0.7529901196047842)
(0.7807394668959586, 0.7673267326732673, 0.7548902874638543)
(0.7873070325900514, 0.7703398558187436, 0.7199261992619926)
-------Macro--------------
(0.7959183673469388, 0.7727498756837393, 0.7529901196047842)
(0.7807394668959586, 0.7673267326732673, 0.7548902874638543)
(0.7873070325900514, 0.7703398558187436, 0.7199261992619926)


In [0]:
# 3 way results
rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=1000)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))

-------Weighted--------------
(0.6554270560783061, 0.5501567299264262, 0.5870600920891088)
(0.6340416652087421, 0.5418967795277382, 0.5837397087937385)
(0.6291157954009651, 0.553473526899492, 0.5803430665529287)
-------Macro--------------
(0.6194239422616149, 0.4991943893562775, 0.43185827653274234)
(0.5988013400383769, 0.48875481927728953, 0.42680607995678493)
(0.5938878796032586, 0.5002426595486532, 0.4424842391334923)


In [0]:
# 5 way results
"""
features = ['q_word_count', 'q_avg_word','q_nouns_vs_words','q_adjectives_vs_words', 'q_verbs_vs_words',
            'r_word_count', 'r_char_count', 'r_avg_word','s_word_count', 's_char_count', 
            's_avg_word', 'r_nouns', 'r_adjectives','r_verbs', 'r_nouns_vs_words', 
            'r_adjectives_vs_words','r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
            's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words','precision', 'recall',
            'F1_score', 'how_flag', 'what_flag', 'why_flag','who_flag', 'which_flag', 'when_flag',
            'where_flag', 'whom_flag','rs_word_diff', 'rs_noun_vs_words_diff', 'rs_verb_vs_words_diff',
            'rs_adjectives_vs_words_diff']
"""

rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=500)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))

-------Weighted--------------
(0.5975543300741246, 0.5062316000025726, 0.4478284373041124)
(0.5771094946981994, 0.45237871522184936, 0.44606815133844496)
(0.5922024854104453, 0.47212219804702815, 0.4554740180244091)
-------Macro--------------
(0.5801716262619241, 0.5098939846678433, 0.3243114086459865)
(0.5459316398271794, 0.4356642985730124, 0.2884080702837859)
(0.578667413168324, 0.4715683975927664, 0.34864931421730516)


In [0]:
# 5 way results
"""
features = ['q_word_count', 'q_avg_word','q_nouns_vs_words','q_adjectives_vs_words', 'q_verbs_vs_words',
            'r_word_count', 'r_char_count', 'r_avg_word','s_word_count', 's_char_count', 
            's_avg_word', 'r_nouns', 'r_adjectives','r_verbs', 'r_nouns_vs_words', 
            'r_adjectives_vs_words','r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
            's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words','precision', 'recall',
            'F1_score','rs_word_diff', 'rs_noun_vs_words_diff', 'rs_verb_vs_words_diff',
            'rs_adjectives_vs_words_diff']
"""

rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=500)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))

-------Weighted--------------
(0.5916790377083754, 0.5055670888176447, 0.4520782782078896)
(0.577686067887289, 0.483909184767496, 0.43897469752297763)
(0.5813742981084858, 0.47724606987934287, 0.4576056101176209)
-------Macro--------------
(0.5707526221243167, 0.5043217352563357, 0.34602126687827894)
(0.5455470796021282, 0.4887758623073668, 0.3238108718685514)
(0.5682442948223396, 0.473901093312204, 0.3438705737019772)


In [0]:
# 5 way results
"""
features = ['q_word_count','q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
            's_word_count', 's_char_count', 's_avg_word', 'q_nouns_vs_words',
            'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
            'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
            'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
            'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
            's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
            's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
            'precision', 'recall', 'F1_score', 'how_flag', 'what_flag', 'why_flag',
            'who_flag', 'which_flag', 'when_flag', 'where_flag', 'whom_flag',
            's_verbs_vs_r_verbs', 's_nouns_vs_r_nouns','s_adjectives_vs_r_adjectives', 
            's_word_count_vs_r_word_count','s_nouns_vs_words_vs_r_nouns_vs_words',
            's_verbs_vs_words_vs_r_verbs_vs_words','s_adjectives_vs_words_vs_r_adjectives_vs_words', 
            'rs_word_diff','rs_noun_vs_words_diff', 'rs_verb_vs_words_diff','rs_adjectives_vs_words_diff']
"""

rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=500)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))

-------Weighted--------------
(0.5871134247785839, 0.4854627701568657, 0.44123438174011215)
(0.5632079095503352, 0.4495824366883241, 0.44191170615570763)
(0.583736584402716, 0.4822659852912088, 0.45231563185107393)
-------Macro--------------
(0.5664102420342532, 0.4907928775159581, 0.3477564461637154)
(0.5374253973272112, 0.4380696989840537, 0.3134606395612832)
(0.5667208920066062, 0.47319181193695625, 0.3558430488998006)


In [0]:
# 5 way results
"""
features = ['q_word_count','q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
            's_word_count', 's_char_count', 's_avg_word', 'q_nouns_vs_words',
            'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
            'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
            'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
            'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
            's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
            's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
            'precision', 'recall', 'F1_score','s_verbs_vs_r_verbs', 's_nouns_vs_r_nouns','s_adjectives_vs_r_adjectives', 
            's_word_count_vs_r_word_count','s_nouns_vs_words_vs_r_nouns_vs_words',
            's_verbs_vs_words_vs_r_verbs_vs_words','s_adjectives_vs_words_vs_r_adjectives_vs_words', 
            'rs_word_diff','rs_noun_vs_words_diff', 'rs_verb_vs_words_diff','rs_adjectives_vs_words_diff']
"""

rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=500)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro'))

-------Weighted--------------
(0.5752945079771895, 0.49695163234731843, 0.4433454258389914)
(0.5700796391129122, 0.4893028365804923, 0.4372459165109884)
(0.569284387867575, 0.5054512623859495, 0.43943732423006077)
-------Macro--------------
(0.557311049146938, 0.5086716926134909, 0.3410167394430168)
(0.5448900466905211, 0.49188928401408666, 0.3197757080952865)
(0.5540001390813345, 0.492459847722649, 0.33254098799455367)


## Grid Run

def run_grid(mtype,grid,n_estimators,criterion,max_depth,max_features,max_leaf_nodes,learning_rate,
            f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes):
    counter = 0
    for est in n_estimators:
        for md in max_depth:
            for mf in max_features:
                for ml in max_leaf_nodes:
                    if mtype == 'RFC':
                        for crt in criterion:
                            counter+=1
                            rfc = RandomForestClassifier(n_estimators=est,criterion=crt,max_depth=md,max_features=mf,max_leaf_nodes=ml)
                            rfc.fit(f_train,y_train)
                            res = run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                            res1 = run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                            tot = {'model':'RFC',
                                   'n_estimators':int(est),
                                   'criterion':str(crt),
                                   'max_depth':int(md),
                                   'max_features':str(mf),
                                   'max_leaf_nodes':int(ml),
                                   'learning_rate':None,
                                   'UA_m_pr':res[0],
                                   'UA_m_re':res[1],
                                   'UA_m_f1':res[2],
                                   'UQ_m_pr':res[3],
                                   'UQ_m_re':res[4],
                                   'UQ_m_f1':res[5],
                                   'UD_m_pr':res[6],
                                   'UD_m_re':res[7],
                                   'UD_m_f1':res[8],
                                   'UA_w_pr':res1[0],
                                   'UA_w_re':res1[1],
                                   'UA_w_f1':res1[2],
                                   'UQ_w_pr':res1[3],
                                   'UQ_w_re':res1[4],
                                   'UQ_w_f1':res1[5],
                                   'UD_w_pr':res1[6],
                                   'UD_w_re':res1[7],
                                   'UD_w_f1':res1[8]
                                  }
                            grid = grid.append(tot,ignore_index=True)
                            if counter%20 == 0:
                                print(counter)
                    elif mtype == 'ETC':
                        for crt in criterion:
                            counter+=1
                            etc = ExtraTreesClassifier(n_estimators=est,criterion=crt,max_depth=md,max_features=mf,max_leaf_nodes=ml)
                            etc.fit(f_train,y_train)
                            res = run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                            res1 = run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                            tot = {'model':'ETC',
                                   'n_estimators':int(est),
                                   'criterion':str(crt),
                                   'max_depth':int(md),
                                   'max_features':str(mf),
                                   'max_leaf_nodes':int(ml),
                                   'learning_rate':None,
                                   'UA_m_pr':res[0],
                                   'UA_m_re':res[1],
                                   'UA_m_f1':res[2],
                                   'UQ_m_pr':res[3],
                                   'UQ_m_re':res[4],
                                   'UQ_m_f1':res[5],
                                   'UD_m_pr':res[6],
                                   'UD_m_re':res[7],
                                   'UD_m_f1':res[8],
                                   'UA_w_pr':res1[0],
                                   'UA_w_re':res1[1],
                                   'UA_w_f1':res1[2],
                                   'UQ_w_pr':res1[3],
                                   'UQ_w_re':res1[4],
                                   'UQ_w_f1':res1[5],
                                   'UD_w_pr':res1[6],
                                   'UD_w_re':res1[7],
                                   'UD_w_f1':res1[8]
                                  }
                            grid = grid.append(tot,ignore_index=True)
                            if counter%20 == 0:
                                print(counter)
                    else:
                        for lr in learning_rate:
                            counter+=1
                            gbc = GradientBoostingClassifier(n_estimators=est,loss='deviance',learning_rate=lr,max_depth=md,max_features=mf,max_leaf_nodes=ml)
                            gbc.fit(f_train,y_train)
                            res = run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                            res1 = run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                            tot = {'model':'GBC',
                                   'n_estimators':int(est),
                                   'criterion':None,
                                   'max_depth':int(md),
                                   'max_features':str(mf),
                                   'max_leaf_nodes':int(ml),
                                   'learning_rate':lr,
                                   'UA_m_pr':res[0],
                                   'UA_m_re':res[1],
                                   'UA_m_f1':res[2],
                                   'UQ_m_pr':res[3],
                                   'UQ_m_re':res[4],
                                   'UQ_m_f1':res[5],
                                   'UD_m_pr':res[6],
                                   'UD_m_re':res[7],
                                   'UD_m_f1':res[8],
                                   'UA_w_pr':res1[0],
                                   'UA_w_re':res1[1],
                                   'UA_w_f1':res1[2],
                                   'UQ_w_pr':res1[3],
                                   'UQ_w_re':res1[4],
                                   'UQ_w_f1':res1[5],
                                   'UD_w_pr':res1[6],
                                   'UD_w_re':res1[7],
                                   'UD_w_f1':res1[8]
                                  }
                            grid = grid.append(tot,ignore_index=True)
                            if counter%50 == 0:
                                print(counter)
    return grid

n_estimators = [100,200,300,400,500]
criterion = ['gini','entropy']
max_depth = [5,10,15,20,25,30]
max_features = ['sqrt','log2']
max_leaf_nodes = [10,25,50,75]
learning_rate = [0.1,0.3,0.01]

grid = pd.DataFrame([],columns=['model','n_estimators','criterion','learning_rate',
                                'UA_m_pr','UA_m_re','UA_m_f1','UQ_m_pr','UQ_m_re','UQ_m_f1','UD_m_pr','UD_m_re','UD_m_f1',
                                'UA_w_pr','UA_w_re','UA_w_f1','UQ_w_pr','UQ_w_re','UQ_w_f1','UD_w_pr','UD_w_re','UD_w_f1'])

In [0]:
def run_grid1(mtype,grid,n_estimators,criterion,learning_rate,f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes):
    counter = 0
    for est in n_estimators:
        if mtype == 'RFC':
            for crt in criterion:
                counter+=1
                print(counter)
                rfc = RandomForestClassifier(n_estimators=est,criterion=crt)
                rfc.fit(f_train,y_train)
                res = run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                res1 = run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                tot = {'model':'RFC',
                       'n_estimators':int(est),
                       'criterion':str(crt),
                       'learning_rate':None,
                       'UA_m_pr':res[0],
                       'UA_m_re':res[1],
                       'UA_m_f1':res[2],
                       'UQ_m_pr':res[3],
                       'UQ_m_re':res[4],
                       'UQ_m_f1':res[5],
                       'UD_m_pr':res[6],
                       'UD_m_re':res[7],
                       'UD_m_f1':res[8],
                       'UA_w_pr':res1[0],
                       'UA_w_re':res1[1],
                       'UA_w_f1':res1[2],
                       'UQ_w_pr':res1[3],
                       'UQ_w_re':res1[4],
                       'UQ_w_f1':res1[5],
                       'UD_w_pr':res1[6],
                       'UD_w_re':res1[7],
                       'UD_w_f1':res1[8]
                      }
                grid = grid.append(tot,ignore_index=True)

        elif mtype == 'ETC':
            for crt in criterion:
                counter+=1
                print(counter)
                etc = ExtraTreesClassifier(n_estimators=est,criterion=crt)
                etc.fit(f_train,y_train)
                res = run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                res1 = run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                tot = {'model':'ETC',
                       'n_estimators':int(est),
                       'criterion':str(crt),
                       'learning_rate':None,
                       'UA_m_pr':res[0],
                       'UA_m_re':res[1],
                       'UA_m_f1':res[2],
                       'UQ_m_pr':res[3],
                       'UQ_m_re':res[4],
                       'UQ_m_f1':res[5],
                       'UD_m_pr':res[6],
                       'UD_m_re':res[7],
                       'UD_m_f1':res[8],
                       'UA_w_pr':res1[0],
                       'UA_w_re':res1[1],
                       'UA_w_f1':res1[2],
                       'UQ_w_pr':res1[3],
                       'UQ_w_re':res1[4],
                       'UQ_w_f1':res1[5],
                       'UD_w_pr':res1[6],
                       'UD_w_re':res1[7],
                       'UD_w_f1':res1[8]
                      }
                grid = grid.append(tot,ignore_index=True)

        else:
            for lr in learning_rate:
                counter+=1
                print(counter)
                gbc = GradientBoostingClassifier(n_estimators=est,loss='deviance',learning_rate=lr)
                gbc.fit(f_train,y_train)
                res = run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                res1 = run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                tot = {'model':'GBC',
                       'n_estimators':int(est),
                       'criterion':None,
                       'learning_rate':lr,
                       'UA_m_pr':res[0],
                       'UA_m_re':res[1],
                       'UA_m_f1':res[2],
                       'UQ_m_pr':res[3],
                       'UQ_m_re':res[4],
                       'UQ_m_f1':res[5],
                       'UD_m_pr':res[6],
                       'UD_m_re':res[7],
                       'UD_m_f1':res[8],
                       'UA_w_pr':res1[0],
                       'UA_w_re':res1[1],
                       'UA_w_f1':res1[2],
                       'UQ_w_pr':res1[3],
                       'UQ_w_re':res1[4],
                       'UQ_w_f1':res1[5],
                       'UD_w_pr':res1[6],
                       'UD_w_re':res1[7],
                       'UD_w_f1':res1[8]
                      }
                grid = grid.append(tot,ignore_index=True)
             
    return grid

In [0]:
n_estimators = [100,200,300,400,500]
criterion = ['gini','entropy']
learning_rate = [0.1,0.2,0.3,0.5]

grid = pd.DataFrame([],columns=['model','n_estimators','criterion','learning_rate',
                                'UA_m_pr','UA_m_re','UA_m_f1','UQ_m_pr','UQ_m_re','UQ_m_f1','UD_m_pr','UD_m_re','UD_m_f1',
                                'UA_w_pr','UA_w_re','UA_w_f1','UQ_w_pr','UQ_w_re','UQ_w_f1','UD_w_pr','UD_w_re','UD_w_f1'])

In [0]:
grid = run_grid1('RFC',grid,n_estimators,criterion,learning_rate,f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes)
grid.to_csv("gdrive/My Drive/ASAG Work/Results/"+str(n_classes)+"way_tree_grid_search_new.csv",sep=',',index=None)
grid = run_grid1('ETC',grid,n_estimators,criterion,learning_rate,f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes)
grid.to_csv("gdrive/My Drive/ASAG Work/Results/"+str(n_classes)+"way_tree_grid_search_new.csv",sep=',',index=None)
grid = run_grid1('GBC',grid,n_estimators,criterion,learning_rate,f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes)
grid.to_csv("gdrive/My Drive/ASAG Work/Results/"+str(n_classes)+"way_tree_grid_search_new.csv",sep=',',index=None)

1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


### Old Results

In [0]:
grid = pd.read_csv("gdrive/My Drive/ASAG Work/Results/5way_tree_grid_search.csv",sep=',')

In [0]:
g1 = grid.sort_values(by=['UA_m_f1','UA_w_f1','UQ_m_f1','UQ_w_f1','UD_m_f1','UD_w_f1'],ascending=False)
g1.iloc[:5,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
24,RFC,500,entropy,NaN,0.582376,0.592112,0.496443,0.505764,0.311870,0.441613
22,RFC,400,entropy,NaN,0.582196,0.597657,0.508901,0.520132,0.315043,0.440064
23,RFC,500,gini,NaN,0.578844,0.591909,0.514204,0.517117,0.320927,0.446049
17,RFC,200,gini,NaN,0.577102,0.593578,0.506222,0.514550,0.313085,0.438474
19,RFC,300,gini,NaN,0.575516,0.589647,0.513223,0.519979,0.303533,0.441129
20,RFC,300,entropy,NaN,0.573535,0.593533,0.515281,0.524838,0.317814,0.445992
18,RFC,200,entropy,NaN,0.572964,0.591256,0.509757,0.512090,0.325381,0.444656
12,GBC,500,NaN,0.1,0.569709,0.576469,0.464728,0.478279,0.316063,0.447306
16,RFC,100,entropy,NaN,0.568882,0.585556,0.499496,0.503325,0.329174,0.444705
21,RFC,400,gini,NaN,0.565498,0.588946,0.510506,0.522731,0.308816,0.443785


In [0]:
g2 = grid.sort_values(by=['UD_m_f1','UD_w_f1','UQ_m_f1','UQ_w_f1','UA_m_f1','UA_w_f1'],ascending=False)
g2.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
3,GBC,200,NaN,0.10,0.533163,0.556563,0.459228,0.481210,0.364422,0.461961
11,GBC,400,NaN,0.01,0.482715,0.513551,0.471959,0.482693,0.359993,0.451158
8,GBC,300,NaN,0.01,0.462408,0.493590,0.445488,0.453131,0.359932,0.447367
14,GBC,500,NaN,0.01,0.491824,0.527743,0.466784,0.476357,0.357434,0.457577
0,GBC,100,NaN,0.10,0.517108,0.540609,0.451802,0.479523,0.341750,0.453206
1,GBC,100,NaN,0.30,0.544924,0.570197,0.451995,0.471771,0.341374,0.459929
6,GBC,300,NaN,0.10,0.561324,0.570076,0.460910,0.477263,0.340614,0.455058
9,GBC,400,NaN,0.10,0.558849,0.572732,0.460966,0.475082,0.336737,0.456677
16,RFC,100,entropy,NaN,0.568882,0.585556,0.499496,0.503325,0.329174,0.444705
4,GBC,200,NaN,0.30,0.546170,0.573552,0.460454,0.479169,0.328757,0.453452


In [0]:
grid = pd.read_csv("gdrive/My Drive/ASAG Work/Results/3way_tree_grid_search.csv",sep=',')

In [0]:
g1 = grid.sort_values(by=['UA_m_f1','UA_w_f1','UQ_m_f1','UQ_w_f1','UD_m_f1','UD_w_f1'],ascending=False)
g1.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
6,RFC,400,gini,NaN,0.631032,0.663615,0.507533,0.556199,0.435268,0.593633
3,RFC,200,entropy,NaN,0.626781,0.657946,0.497607,0.550042,0.430351,0.587169
2,RFC,200,gini,NaN,0.626515,0.660540,0.502527,0.552949,0.428694,0.584765
7,RFC,400,entropy,NaN,0.624369,0.658787,0.495816,0.549027,0.435443,0.593913
0,RFC,100,gini,NaN,0.623984,0.658715,0.496428,0.547091,0.431081,0.587824
5,RFC,300,entropy,NaN,0.622783,0.655971,0.495136,0.548389,0.434387,0.592597
9,RFC,500,entropy,NaN,0.622328,0.657895,0.485880,0.540578,0.437756,0.594161
4,RFC,300,gini,NaN,0.619829,0.654432,0.495905,0.548085,0.439123,0.598578
11,ETC,100,entropy,NaN,0.616931,0.651503,0.494389,0.542584,0.434433,0.594401
8,RFC,500,gini,NaN,0.616885,0.653341,0.505867,0.556426,0.429571,0.585732


In [0]:
g2 = grid.sort_values(by=['UD_m_f1','UD_w_f1','UQ_m_f1','UQ_w_f1','UA_m_f1','UA_w_f1'],ascending=False)
g2.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
20,GBC,100,NaN,0.10,0.572167,0.615352,0.490411,0.547519,0.442560,0.589050
31,GBC,400,NaN,0.01,0.523924,0.588001,0.417325,0.498173,0.440172,0.590715
4,RFC,300,gini,NaN,0.619829,0.654432,0.495905,0.548085,0.439123,0.598578
28,GBC,300,NaN,0.01,0.505613,0.575037,0.412324,0.495920,0.438645,0.588913
34,GBC,500,NaN,0.01,0.536036,0.593903,0.426078,0.502815,0.438594,0.589848
18,ETC,500,gini,NaN,0.606863,0.642934,0.483198,0.533370,0.438496,0.600284
23,GBC,200,NaN,0.10,0.587885,0.628948,0.495741,0.550982,0.437842,0.584860
9,RFC,500,entropy,NaN,0.622328,0.657895,0.485880,0.540578,0.437756,0.594161
12,ETC,200,gini,NaN,0.602257,0.638784,0.483525,0.533304,0.437208,0.598490
19,ETC,500,entropy,NaN,0.610607,0.646342,0.476759,0.530369,0.436943,0.598406


In [0]:
grid = pd.read_csv("gdrive/My Drive/ASAG Work/Results/2way_tree_grid_search.csv",sep=',')

In [0]:
g1 = grid.sort_values(by=['UA_m_f1','UA_w_f1','UQ_m_f1','UQ_w_f1','UD_m_f1','UD_w_f1'],ascending=False)
g1.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
5,RFC,300,entropy,NaN,0.798629,0.798629,0.769770,0.769770,0.759693,0.759693
3,RFC,200,entropy,NaN,0.798617,0.798617,0.773679,0.773679,0.760506,0.760506
6,RFC,400,gini,NaN,0.796217,0.796217,0.768844,0.768844,0.761529,0.761529
7,RFC,400,entropy,NaN,0.795160,0.795160,0.773253,0.773253,0.760226,0.760226
32,GBC,500,NaN,0.1,0.794850,0.794850,0.752284,0.752284,0.716798,0.716798
8,RFC,500,gini,NaN,0.794449,0.794449,0.776376,0.776376,0.758941,0.758941
29,GBC,400,NaN,0.1,0.793814,0.793814,0.761290,0.761290,0.721245,0.721245
2,RFC,200,gini,NaN,0.793761,0.793761,0.778057,0.778057,0.758938,0.758938
0,RFC,100,gini,NaN,0.792808,0.792808,0.775449,0.775449,0.752169,0.752169
12,ETC,200,gini,NaN,0.791557,0.791557,0.767096,0.767096,0.767327,0.767327


In [0]:
g2 = grid.sort_values(by=['UD_m_f1','UD_w_f1','UQ_m_f1','UQ_w_f1','UA_m_f1','UA_w_f1'],ascending=False)
g2.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
12,ETC,200,gini,NaN,0.791557,0.791557,0.767096,0.767096,0.767327,0.767327
9,RFC,500,entropy,NaN,0.789700,0.789700,0.775984,0.775984,0.765487,0.765487
17,ETC,400,entropy,NaN,0.789199,0.789199,0.765321,0.765321,0.764599,0.764599
14,ETC,300,gini,NaN,0.787719,0.787719,0.770370,0.770370,0.764323,0.764323
13,ETC,200,entropy,NaN,0.782456,0.782456,0.775248,0.775248,0.764023,0.764023
19,ETC,500,entropy,NaN,0.790576,0.790576,0.766420,0.766420,0.763691,0.763691
15,ETC,300,entropy,NaN,0.790252,0.790252,0.769307,0.769307,0.763127,0.763127
16,ETC,400,gini,NaN,0.790576,0.790576,0.770069,0.770069,0.762900,0.762900
6,RFC,400,gini,NaN,0.796217,0.796217,0.768844,0.768844,0.761529,0.761529
22,GBC,100,NaN,0.01,0.762490,0.762490,0.765069,0.765069,0.761453,0.761453


### New Results

In [0]:
grid = pd.read_csv("gdrive/My Drive/ASAG Work/Results/5way_tree_grid_search_new.csv",sep=',')

In [0]:
g1 = grid.sort_values(by=['UA_m_f1','UA_w_f1','UQ_m_f1','UQ_w_f1','UD_m_f1','UD_w_f1'],ascending=False)
g1.iloc[:5,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
32,GBC,400,NaN,0.1,0.562963,0.576972,0.493172,0.504331,0.323778,0.443218
9,RFC,500,entropy,NaN,0.561186,0.582587,0.506977,0.507537,0.352543,0.445641
3,RFC,200,entropy,NaN,0.559914,0.581344,0.490129,0.485662,0.376537,0.445723
7,RFC,400,entropy,NaN,0.558236,0.577683,0.499131,0.495565,0.362374,0.440792
0,RFC,100,gini,NaN,0.557148,0.579647,0.509435,0.498993,0.339967,0.448051


In [0]:
g2 = grid.sort_values(by=['UD_m_f1','UD_w_f1','UQ_m_f1','UQ_w_f1','UA_m_f1','UA_w_f1'],ascending=False)
g2.iloc[:5,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
3,RFC,200,entropy,NaN,0.559914,0.581344,0.490129,0.485662,0.376537,0.445723
7,RFC,400,entropy,NaN,0.558236,0.577683,0.499131,0.495565,0.362374,0.440792
1,RFC,100,entropy,NaN,0.548738,0.562655,0.497640,0.488788,0.361001,0.441896
4,RFC,300,gini,NaN,0.547836,0.573275,0.513534,0.506167,0.353283,0.437693
9,RFC,500,entropy,NaN,0.561186,0.582587,0.506977,0.507537,0.352543,0.445641


In [0]:
grid = pd.read_csv("gdrive/My Drive/ASAG Work/Results/3way_tree_grid_search_new.csv",sep=',')

In [0]:
g1 = grid.sort_values(by=['UA_m_f1','UA_w_f1','UQ_m_f1','UQ_w_f1','UD_m_f1','UD_w_f1'],ascending=False)
g1.iloc[:5,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
5,RFC,300,entropy,NaN,0.628811,0.662679,0.487101,0.541225,0.428195,0.583875
8,RFC,500,gini,NaN,0.622969,0.658154,0.494730,0.546456,0.430822,0.587549
1,RFC,100,entropy,NaN,0.622555,0.652688,0.505894,0.556552,0.432750,0.585476
3,RFC,200,entropy,NaN,0.620663,0.657915,0.494675,0.545534,0.428842,0.586407
4,RFC,300,gini,NaN,0.619589,0.657572,0.488552,0.541901,0.424971,0.581234


In [0]:
g2 = grid.sort_values(by=['UD_m_f1','UD_w_f1','UQ_m_f1','UQ_w_f1','UA_m_f1','UA_w_f1'],ascending=False)
g2.iloc[:5,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
28,GBC,300,NaN,0.1,0.598770,0.635091,0.512490,0.564850,0.450088,0.596763
24,GBC,200,NaN,0.1,0.593551,0.631448,0.488029,0.540729,0.449525,0.599580
20,GBC,100,NaN,0.1,0.567054,0.609825,0.480639,0.535441,0.448059,0.600406
21,GBC,100,NaN,0.2,0.586242,0.626883,0.478927,0.532537,0.444581,0.594488
36,GBC,500,NaN,0.1,0.594958,0.631214,0.509175,0.561930,0.444180,0.587112


In [0]:
grid = pd.read_csv("gdrive/My Drive/ASAG Work/Results/2way_tree_grid_search_new.csv",sep=',')

In [0]:
g1 = grid.sort_values(by=['UA_m_f1','UA_w_f1','UQ_m_f1','UQ_w_f1','UD_m_f1','UD_w_f1'],ascending=False)
g1.iloc[:5,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
30,GBC,100,NaN,0.1,0.799666,0.799666,0.771646,0.771646,0.731904,0.731904
31,GBC,100,NaN,0.2,0.793597,0.793597,0.774793,0.774793,0.740206,0.740206
1,RFC,100,entropy,NaN,0.793191,0.793191,0.771229,0.771229,0.747931,0.747931
35,GBC,200,NaN,0.2,0.792899,0.792899,0.776739,0.776739,0.733273,0.733273
12,RFC,200,gini,NaN,0.792869,0.792869,0.774613,0.774613,0.747306,0.747306


In [0]:
g2 = grid.sort_values(by=['UD_m_f1','UD_w_f1','UQ_m_f1','UQ_w_f1','UA_m_f1','UA_w_f1'],ascending=False)
g2.iloc[:5,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
25,ETC,300,entropy,NaN,0.787252,0.787252,0.769990,0.769990,0.758725,0.758725
27,ETC,400,entropy,NaN,0.781116,0.781116,0.768924,0.768924,0.757157,0.757157
13,RFC,200,entropy,NaN,0.791171,0.791171,0.766551,0.766551,0.756279,0.756279
0,RFC,100,gini,NaN,0.781277,0.781277,0.768849,0.768849,0.756157,0.756157
17,RFC,400,entropy,NaN,0.789787,0.789787,0.768463,0.768463,0.755778,0.755778


# NN model

In [0]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

In [0]:
def apply_normalize(f,method=1):
    if method == 1:
        # min max
        f.iloc[:,:-8] = (f.iloc[:,:-8]-f.iloc[:,:-8].min())/(f.iloc[:,:-8].max()-f.iloc[:,:-8].min())
    elif method == 2:
        # z score
        f.iloc[:,:-8] = (f.iloc[:,:-8]-f.iloc[:,:-8].mean())/f.iloc[:,:-8].std()
    return f

In [0]:
def build_keras_model1(features,n_classes):
    inp = tf.keras.layers.Input((len(features),))
    x = tf.keras.layers.Dense(128,activation='relu')(inp)
    x = tf.keras.layers.Dense(128,activation='relu')(x)
    x = tf.keras.layers.Dense(128,activation='relu')(x)
    x = tf.keras.layers.Dense(128,activation='relu')(x)
    out = tf.keras.layers.Dense(n_classes,activation='softmax')(x)
    model = tf.keras.models.Model(inputs=[inp],outputs=[out],name='DNN')
    return model

## Normal Run

### New Features

In [0]:
def run_model(f_train,y_train,features,n_classes,norm):
    if norm:
        feat_train = apply_normalize(f_train,norm)
    else:
        feat_train = f_train
    keras_model = build_keras_model1(features,n_classes)
    tf.keras.backend.clear_session()
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(keras_model, 
                                                  strategy=tf.contrib.tpu.TPUDistributionStrategy(
                                                      tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

    tpu_model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    tpu_model.fit([feat_train],y_train,epochs=50,batch_size=32)#,callbacks=[tf.keras.callbacks.EarlyStopping('loss',0.1,10)])
    tpu_model.save_weights('./model.h5', overwrite=True)

In [0]:
def collect_results(f_test1,y_test1,f_test2,y_test2,f_test3,y_test3,features,n_classes,method,norm):

    model = build_keras_model1(features,n_classes)
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    model.load_weights("./model.h5")

    feat_test1 = apply_normalize(f_test1,norm)
    feat_test2 = apply_normalize(f_test2,norm)
    feat_test3 = apply_normalize(f_test3,norm)
#     feat_test1 = f_test1
#     feat_test2 = f_test2
#     feat_test3 = f_test3
    
    results = []
    probs = model.predict(feat_test1)
    y_pred = np.argmax(probs,axis=1)
    if n_classes == 2:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    probs = model.predict(feat_test2)
    y_pred = np.argmax(probs,axis=1)
    if n_classes == 2:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    probs = model.predict(feat_test3)
    y_pred = np.argmax(probs,axis=1)
    if n_classes == 2:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    return (results[2],results[5],results[8])
    return results

In [0]:
norm = 1 # 0 or 1 or 2 
run_model(f_train,y_train,features,n_classes,norm)
print("-------Weighted--------------")
print(collect_results(f_test1,y_test1,f_test2,y_test2,f_test3,y_test3,features,n_classes,'weighted',norm))
print("-------Macro--------------")
print(collect_results(f_test1,y_test1,f_test2,y_test2,f_test3,y_test3,features,n_classes,'macro',norm))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.0.182.146:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 14735603753082157386)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4432702292124324862)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 8645164433582854863)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 386353438747288701)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14217556930645895859)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
norm = 2 # 0 or 1 or 2 
run_model(f_train,y_train,features,n_classes,norm)
print("-------Weighted--------------")
print(collect_results(f_test1,y_test1,f_test2,y_test2,f_test3,y_test3,features,n_classes,'weighted',norm))
print("-------Macro--------------")
print(collect_results(f_test1,y_test1,f_test2,y_test2,f_test3,y_test3,features,n_classes,'macro',norm))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.0.182.146:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 14735603753082157386)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4432702292124324862)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 8645164433582854863)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 386353438747288701)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14217556930645895859)
INFO:tensorflow:*** Available Device: _DeviceA

### Old Features

In [0]:
def build_keras_dnn(var,units,act,drop,regu,regu_val):
    
    if regu == 'l1':
        re = tf.keras.regularizers.l1(regu_val)
    elif regu == 'l2':
        re = tf.keras.regularizers.l2(regu_val)
    else:
        re = tf.keras.regularizers.l1_l2(l1=regu_val,l2=regu_val)
    
    inp1 = tf.keras.layers.Input((len(features),))
    q_embed = tf.keras.layers.Input((4096,))
    r_embed = tf.keras.layers.Input((4096,))
    s_embed = tf.keras.layers.Input((4096,))
    
#     q_embed = tf.keras.layers.Dense(300)(qinp)
#     r_embed = tf.keras.layers.Dense(300)(rinp)
#     s_embed = tf.keras.layers.Dense(300)(sinp)
    
    add_qr = tf.keras.layers.Add()([q_embed,r_embed])
    add_rs = tf.keras.layers.Add()([r_embed,s_embed])
    sub_rs = tf.keras.layers.Subtract()([s_embed,r_embed])
    sub_srq = tf.keras.layers.Subtract()([s_embed,add_qr])
    mul_rs = tf.keras.layers.Multiply()([r_embed,s_embed])
    
    if var == 1:
        concat = tf.keras.layers.Concatenate()([add_qr,add_rs,sub_rs,sub_srq,mul_rs,inp1])

        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(concat)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        out = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x)
    
    if var == 2:
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(inp1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        
        concat = tf.keras.layers.Concatenate()([add_qr,add_rs,sub_rs,sub_srq,mul_rs])
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(concat)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        
        concat = tf.keras.layers.Concatenate()([x1,x2])
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(concat)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        out = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x)
        
    if var == 3:
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(inp1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x1)
        
        concat = tf.keras.layers.Concatenate()([add_qr,add_rs,sub_rs,sub_srq,mul_rs])
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(concat)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x2)
        
        x = tf.keras.layers.Multiply()([x1,x2])
        out = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x)
        
    model = tf.keras.models.Model(inputs=[inp1,q_embed,r_embed,s_embed],outputs=[out],name='DNN_Embed_Features')
    return model

In [0]:
def run_model(norm_val,var,f_train,q_train,r_train,s_train,y_train):
    
#     feat_train = apply_normalize(f_train,norm_val)
    feat_train = f_train
    keras_model = build_keras_dnn(var,64,'relu',0.2,'l1',0.01)
    tf.keras.backend.clear_session()
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(keras_model, 
                                                  strategy=tf.contrib.tpu.TPUDistributionStrategy(
                                                      tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

    tpu_model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    tpu_model.fit([feat_train,q_train,r_train,s_train],y_train,epochs=25,batch_size=64,callbacks=[tf.keras.callbacks.EarlyStopping('loss',0.1,10)])
    tpu_model.save_weights('./model.h5', overwrite=True)

In [0]:
def collect_results(norm_val,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,n_classes,method):

    model = build_keras_dnn(var,64,'relu',0.2,'l1',0.01)
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    model.load_weights("./model.h5")

#     feat_test1 = apply_normalize(f_test1,norm_val)
#     feat_test2 = apply_normalize(f_test2,norm_val)
#     feat_test3 = apply_normalize(f_test3,norm_val)
    feat_test1 = f_test1
    feat_test2 = f_test2
    feat_test3 = f_test3
    
    results = []
    probs = model.predict([feat_test1,q_test1,r_test1,s_test1])
    y_pred = np.argmax(probs,axis=1)
    if n_classes == 2:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    probs = model.predict([feat_test2,q_test2,r_test2,s_test2])
    y_pred = np.argmax(probs,axis=1)
    if n_classes == 2:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    probs = model.predict([feat_test3,q_test3,r_test3,s_test3])
    y_pred = np.argmax(probs,axis=1)
    if n_classes == 2:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    return results

In [0]:
var = 1
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 1
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 1
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'macro'))

In [0]:
var = 2
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 2
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 2
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 3
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 3
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 3
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'macro'))

## Grid Run

In [0]:
def grid_nn(normalize,variant,activation,units,dropout,regularize,regu_value,learning_rate,optimizer,grid,
           q_train,r_train,s_train,feat_train,y_train,q_test1,r_test1,s_test1,feat_test1,y_test1,
            q_test2,r_test2,s_test2,feat_test2,y_test2,q_test3,r_test3,s_test3,feat_test3,y_test3):
    for norm in normalize:
        for var in variant:
            for act in activation:
                for u in units:
                    for d in dropout:
                        for r in regularize:
                            for rv in regu_value:
                                for lr in learning_rate:
                                    for opt in optimizer:
                                        if opt == 'adam':
                                            optim = tf.train.AdamOptimizer(learning_rate=lr)
                                        elif opt == 'sgd':
                                            optim = tf.train.GradientDescentOptimizer(learning_rate=lr)
                                        elif opt == 'adadelta':
                                            optim = tf.train.AdadeltaOptimizer(learning_rate=lr)
                                        elif opt == 'adagrad':
                                            optim = tf.train.AdagradOptimizer(learning_rate=lr)
                                        elif opt == 'rmsprop':
                                            optim = tf.train.RMSPropOptimizer(learning_rate=lr)
                                        feat_train = apply_normalize(f_train,norm)
                                        feat_test1 = apply_normalize(f_test1,norm)
                                        feat_test2 = apply_normalize(f_test2,norm)
                                        feat_test3 = apply_normalize(f_test3,norm)
                                        keras_model = build_keras_dnn(var,u,act,d,r,rv)
                                        keras_model.summary()
                                        tf.keras.backend.clear_session()
                                        tpu_model = tf.contrib.tpu.keras_to_tpu_model(keras_model, 
                                                                                      strategy=tf.contrib.tpu.TPUDistributionStrategy(
                                                                                          tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

                                        tpu_model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
                                        callback = tf.keras.callbacks.EarlyStopping(monitor='loss',min_delta=0.3,patience=3)
                                        tpu_model.fit([feat_train,q_train,r_train,s_train],y_train,epochs=25,batch_size=64,callbacks=[callback])
                                        tpu_model.save_weights('./model.h5', overwrite=True)
                                        # results
                                        res = []
                                        model = build_keras_dnn(var)
                                        model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
                                        model.load_weights("./model.h5")
                                        #UA
                                        probs = model.predict([feat_test1,q_test1,r_test1,s_test1])
                                        y_pred = np.argmax(probs,axis=1)
                                        f1 = f1_score(y_test1,y_pred)
                                        pr = precision_score(y_test1,y_pred)
                                        re = recall_score(y_test1,y_pred)
                                        res.extend([pr,re,f1])
                                        #UQ
                                        probs = model.predict([feat_test2,q_test2,r_test2,s_test2])
                                        y_pred = np.argmax(probs,axis=1)
                                        f1 = f1_score(y_test2,y_pred)
                                        pr = precision_score(y_test2,y_pred)
                                        re = recall_score(y_test2,y_pred)
                                        res.extend([pr,re,f1])
                                        #UD
                                        probs = model.predict([feat_test3,q_test3,r_test3,s_test3])
                                        y_pred = np.argmax(probs,axis=1)
                                        f1 = f1_score(y_test3,y_pred)
                                        pr = precision_score(y_test3,y_pred)
                                        re = recall_score(y_test3,y_pred)
                                        res.extend([pr,re,f1])
                                        tot = {'variant':var,
                                               'normalize':norm,
                                               'activation':act,
                                               'units':u,
                                               'dropout':d,
                                               'regularizer':r,
                                               'regularizer_value':rv,
                                               'learning_rate':lr,
                                               'optimizer':opt,
                                               'UA_pr':res[0],
                                               'UA_re':res[1],
                                               'UA_f1':res[2],
                                               'UQ_pr':res[3],
                                               'UQ_re':res[4],
                                               'UQ_f1':res[5],
                                               'UD_pr':res[6],
                                               'UD_re':res[7],
                                               'UD_f1':res[8]}
                                        grid.append(tot,ignore_index=True)
    return grid

In [0]:
normalize = [-1,0,1]
variant = [1,2,3]
activation = ['tanh','sigmoid','relu','prelu','lrelu','elu']
units = [16,32,64,128]
dropout = [0.0,0.1,0.2]
regularize = ['l1','l2','l1_l2']
regu_value = [0.1,0.001]
learning_rate = [0.001,0.005,0.01,0.05]
optimizer = ['adam','sgd','adadelta','adagrad','rmsprop']

grid = pd.DataFrame([],columns=['variant','normalize','activation','units','dropout','regularizer',
                                'regularizer_value','learning_rate','optimizer','UA_pr','UA_re','UA_f1',
                                'UQ_pr','UQ_re','UQ_f1','UD_pr','UD_re','UD_f1'])

In [0]:
grid = grid_nn(normalize,variant,activation,units,dropout,regularize,regu_value,learning_rate,optimizer,grid,
           q_train,r_train,s_train,f_train,y_train,q_test1,r_test1,s_test1,f_test1,y_test1,
            q_test2,r_test2,s_test2,f_test2,y_test2,q_test3,r_test3,s_test3,f_test3,y_test3)

NameError: ignored

# Feature Selected

In [0]:
feat_sel = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/Feature Selection.csv")

In [0]:
feat_sel

,Unnamed: 0,"RFC,400,gini","RFC,200,entropy","RFC,200,gini","RFC,400,entropy","RFC,100,gini","GBC,100,0.1","GBC,400,0.01","RFC,300,gini","GBC,300,0.01","GBC,500,0.01"
0,0,Row_no,Row_no,Row_no,Row_no,Row_no,recall,recall,Row_no,recall,recall
1,1,F1_score,F1_score,F1_score,F1_score,F1_score,Row_no,Row_no,recall,Row_no,Row_no
2,2,recall,s_char_count,recall,s_char_count,s_char_count,r_avg_word,r_avg_word,s_char_count,r_char_count,r_avg_word
3,3,s_char_count,recall,s_char_count,recall,recall,r_char_count,r_char_count,F1_score,r_avg_word,r_char_count
4,4,s_avg_word,s_avg_word,s_avg_word,s_avg_word,s_avg_word,q_nouns_vs_words,F1_score,s_avg_word,F1_score,q_nouns_vs_length
5,5,r_char_count,r_char_count,r_char_count,r_char_count,r_char_count,F1_score,q_nouns_vs_length,r_char_count,q_nouns_vs_length,F1_score
6,6,s_nouns_vs_length,s_nouns_vs_length,s_nouns_vs_length,s_nouns_vs_length,s_nouns_vs_length,r_nouns_vs_words,r_nouns,s_nouns_vs_length,r_nouns,r_nouns
7,7,r_avg_word,r_avg_word,r_avg_word,r_avg_word,r_avg_word,s_avg_word,r_word_count,r_avg_word,r_word_count,r_nouns_vs_words
8,8,s_verbs_vs_length,s_verbs_vs_length,s_verbs_vs_length,s_verbs_vs_length,s_verbs_vs_length,q_nouns_vs_length,r_nouns_vs_words,s_verbs_vs_length,r_nouns_vs_words,r_word_count
9,9,precision,precision,precision,precision,precision,r_nouns,s_avg_word,precision,s_avg_word,s_avg_word


In [0]:
tmp_df = data[['ref_answer','stu_answer']]
tmp_df.to_csv("tmp.csv",sep=',',index=None)

In [0]:
import csv
#Open file with "w"(write), the file will create automatically.
file = open("gdrive/My Drive/ASAG Work/pairs.txt", "w")
#Open CSV file to read CSV, note: reading and write file should be under "with"
with open('tmp.csv') as csvFile:
    #Read CSV
    readCsv = csv.reader(csvFile)
    for row in readCsv:
        #Get Values and manupilate in the file.write
        Id = row[0]
        Id1 = row[1]
        #Write CSV you need format it to string if the value is an int
        file.write(Id+'\t'+Id1+'\n')
#You Must Close the FIle after writing it.
file.close()

In [0]:
output=open('gdrive/My Drive/ASAG Work/pairs.txt','w')

with open('gdrive/My Drive/Microsoft AI/Junk/Sample Data/Sample_Data.csv',"rt", encoding='ascii') as f:
  for row in tempf:
    output.write(row)    